In [1]:
import torch
import torch.nn as nn
import numpy as np
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [4]:
# prepare the data
bc = datasets.load_breast_cancer()
X, y = bc.data, bc.target
n_samples, n_features = X.shape
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=1234)

sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

X_train = torch.tensor(X_train.astype(np.float32))
X_test = torch.tensor(X_test.astype(np.float32))
y_train = torch.tensor(y_train.astype(np.float32))
y_test = torch.tensor(y_test.astype(np.float32))

y_train = y_train.view(-1, 1)
y_test = y_test.view(-1, 1)

In [10]:
class Model(nn.Module):

    def __init__(self, n_input_features):
        super(Model, self).__init__()
        self.linear = nn.Linear(n_input_features, 1)

    def forward(self, x):
        return torch.sigmoid(self.linear(x))

In [11]:
model = Model(n_features)
criterion = nn.BCELoss()
learning_rate = .01
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
num_epoths = 100

for epoth in range(num_epoths):
    y_pred = model(X_train)
    loss = criterion(y_pred, y_train)
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

    if (epoth + 1) % 10 == 0:
        print(f'epoth: {epoth+1}, loss: {loss.item()}')

# evaluate
with torch.no_grad():
    y_predicted = model(X_test)
    y_predicted_cls = y_predicted.round()
    acc = y_predicted_cls.eq(y_test).sum() / float(y_test.shape[0])
    print(f'accuracy = {acc:.4f}')

epoth: 10, loss: 0.5191121101379395
epoth: 20, loss: 0.43686357140541077
epoth: 30, loss: 0.38275817036628723
epoth: 40, loss: 0.34422752261161804
epoth: 50, loss: 0.31519797444343567
epoth: 60, loss: 0.2924060523509979
epoth: 70, loss: 0.2739439010620117
epoth: 80, loss: 0.2586204707622528
epoth: 90, loss: 0.2456524819135666
epoth: 100, loss: 0.23450271785259247
accuracy = 0.9211
